In [1]:
import requests
import warnings
warnings.filterwarnings("ignore")

In [67]:
# this method fetch data from the backend; triggered by url call
# fetch data allows you to qurey from the body argument

def fetch_data(body, index_name):
    url = 'http://127.0.0.1:9090/query-es'
    response = requests.post(url, json={'body': body, 'index': index_name})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to fetch data: Status code {response.status_code}"


# this function return the WHOLE es index
# be careful with large data set
# 中文再说一遍，这个function把整个表都读进来，如果数据很大会很慢
# 为了避免内存爆炸，考虑使用上面的函数写query

def fetch_all_data(index: str):
    url = 'http://127.0.0.1:9090/fetch-all'
    response = requests.post(url, json={'index': index}, headers={"Content-Type": "application/json"})
    if response.status_code == 200:
        return response.json()  # Make sure to return the parsed JSON
    else:
        return f"Failed to fetch data: Status code {response.status_code}"



In [68]:
# testing querying the first 5 rows from the sensor index
query = {"size": 10, "query": {"match_all": {}}}
index = 'sensor'
test = fetch_data(index)

TypeError: fetch_data() missing 1 required positional argument: 'index_name'

In [69]:
test

[{'_id': 'DGhlYI8BAp0N_qlVe4j5',
  '_index': 'sensor',
  '_routing': 'sensor',
  '_score': 1.0,
  '_source': {'battery': 3,
   'date': '2023-02-26',
   'dev_id': 'r712-a7f7',
   'humidity': 71.09,
   'location': {'lat': -37.8030396, 'lon': 144.965898},
   'temperature': 20.16,
   'time': '05:37:18'}},
 {'_id': 'DWhlYI8BAp0N_qlVfYiw',
  '_index': 'sensor',
  '_routing': 'sensor',
  '_score': 1.0,
  '_source': {'battery': 3,
   'date': '2023-02-26',
   'dev_id': 'r712-4e85',
   'humidity': 71.79,
   'location': {'lat': -37.8024211, 'lon': 144.9663054},
   'temperature': 19.96,
   'time': '05:41:09'}},
 {'_id': 'DmhlYI8BAp0N_qlVfogs',
  '_index': 'sensor',
  '_routing': 'sensor',
  '_score': 1.0,
  '_source': {'battery': 3,
   'date': '2023-02-26',
   'dev_id': 'r712-a7f7',
   'humidity': 68.32,
   'location': {'lat': -37.8030396, 'lon': 144.965898},
   'temperature': 20.11,
   'time': '07:37:19'}},
 {'_id': 'D2hlYI8BAp0N_qlVfohV',
  '_index': 'sensor',
  '_routing': 'sensor',
  '_score':

In [70]:
# reading the fix data into pd.dataframe
import pandas as pd
def es_into_df(es_outcome: list):
    data = []
    for document in es_outcome:
        if '_source' in document:
            data.append(document['_source'])
    df = pd.DataFrame(data)
    return df

In [71]:
es_into_df(test)

,battery,date,dev_id,humidity,location,temperature,time
0,3,2023-02-26,r712-a7f7,71.09,"{'lat': -37.8030396, 'lon': 144.965898}",20.16,05:37:18
1,3,2023-02-26,r712-4e85,71.79,"{'lat': -37.8024211, 'lon': 144.9663054}",19.96,05:41:09
2,3,2023-02-26,r712-a7f7,68.32,"{'lat': -37.8030396, 'lon': 144.965898}",20.11,07:37:19
3,3,2023-02-26,r712-a7f8,74.18,"{'lat': -37.8027955, 'lon': 144.9657785}",19.14,09:33:40
4,3,2023-02-26,r712-a7f9,76.76,"{'lat': -37.8026473, 'lon': 144.9661236}",18.93,10:28:31
5,3,2023-02-26,r712-a7f8,76.93,"{'lat': -37.8027955, 'lon': 144.9657785}",18.90,10:33:40
6,3,2023-02-26,r712-a7f8,79.08,"{'lat': -37.8027955, 'lon': 144.9657785}",18.59,11:33:40
7,3,2023-02-26,r712-a7f9,80.38,"{'lat': -37.8026473, 'lon': 144.9661236}",18.24,12:28:32
8,3,2023-02-26,r712-a7f7,79.62,"{'lat': -37.8030396, 'lon': 144.965898}",18.39,12:37:20
9,3,2023-02-26,r712-a7f8,81.77,"{'lat': -37.8027955, 'lon': 144.9657785}",18.17,14:33:41
